In [23]:
import os
import tensorflow.keras as keras
import numpy as np
import music21 as m21
import json
import random
from prep_functions import SEQUENCE_LENGTH

In [24]:
# Variables

# 17 seeds
china_seeds = ['67 _ 67 _ 62 _ 62',
               '67 _ 67 _ 64 _ 72 _ 69 _ 67 _', 
               '69 _ 76 _ 74 _ 74 72',
               '67 _ _ 64 62 _ _ 64 67 _ _',
               '74 _ _ 72 74 _ 72 _ 72 69 _ _', 
               '72 _ _ 72 72 _ 69 _ 67 _ _ _',
               '69 _ 69 67 69 _ 76 _ 74 _ _ _',
               '64 _ _ _ 67 _ _ 64 64 _ 62 _',
               '67 _ _ _ 67 _ 76 _ 76 _',
               '74 _ 72 74 76 _ 74 _',
               '72 _ 72 _ 74 _ 76 _ _ 76',
               '69 _ 67 _ 69 _ _ _ 69 _',
               '72 _ _ 69 67 _ 72 _',
               '69 _ 69 67 65 67 69 72',
               '67 _ _ _ 65 _ 64 _ 69 _',
               '67 _ _ _ 64 _ 62 _ 67 _ _ _',
               '79 76 _ 74 74 _ 76 _'
              ]

# 37 seeds
deutschl_seeds = ['55 _ 60 _ 60 _ 60 _ 64 _',
                  '67 _ _ _ 64 _ 64 _ 64 _',
                  '60 _ _ _ _ _ _ _ 62 _ _ _ _ _ _ _ 60 _ _ _ _ _ _ _',
                  '76 _ _ _ 76 _ _ _ 76 _ _ _ 76 _ _ _',
                  '60 _ _ _ 60 _ _ _ 60 _ _ _ 62 _ _ _',
                  '76 _ _ _ 69 _ _ _ 69 _ _ _ 69 _ _ _',
                  '55 _ 60 _ 64 _ 64 _ _ _',
                  '55 _ 60 _ 60 _ 60 _ 60 _',
                  '72 _ _ _ _ _ _ _ 72 _ _ _ 74 _ _ _',
                  '72 _ _ _ 74 _ 72 _ 71 _ 69 _',
                  '65 _ 64 _ _ _ 65 _ 65 _ _ _',
                  '55 _ 53 _ 52 _ _ _ 55 _ _ _',
                  '60 _ _ _ 64 _ _ _ 67 _ 64 _',
                  '55 _ _ _ 64 _ _ 62 60 _ 60 _',
                  '55 _ _ 57 55 _ 52 _ _ _ _ _ 55 _ _ 57',
                  '55 _ 55 _ 64 _ _ _ _ _ 62 _',
                  '64 _ 69 _ 69 _ 71 _ 72 _',
                  '55 _ 60 _ 55 _ 60 _ 64 _',
                  '55 _ _ _ 60 _ _ _ 62 _ _ _ 64 _',
                  '60 _ 64 _ 64 62 60 _ _ _ _ _',
                  '55 _ 55 _ 60 _ 60 _ 60 _',
                  '69 _ _ _ 76 _ _ _ 74 _ _ 72',
                  '60 _ 60 _ _ _ 64 _ 67 _ _ _',
                  '60 _ 60 _ 64 _ 67 _ _ _',
                  '64 _ _ _ 64 _ _ _ 64 _ 62 64',
                  '64 _ 60 _ 60 _ 57 _ 57 _',
                  '60 _ 64 _ 67 _ _ _ 67 _ _ _',
                  '67 _ _ _ _ _ 69 _ 67 _ 65 _',
                  '55 _ 60 _ 60 _ 55 _ 55 _',
                  '55 _ _ 53 52 _ _ _ 48 _ _ _',
                  '64 _ _ 62 62 _ _ _ 60 _ _ _',
                  '55 _ 57 _ 55 _ 54 _ 55 _ 60 _ _',
                  '67 _ _ _ _ _ 69 _ 67 _ 65 _',
                  '60 _ 60 _ 60 _ 64 _ 60 _',
                  '60 _ 60 _ 64 _ 67 _ 72 _',
                  '55 _ 60 _ _ 62 64 _ 65 _',
                  '60 _ 60 _ 64 65 67 _ 62',
                 ]

In [54]:
class MelodyGenerator:
    
    def __init__(self, dataset, model_filename):
        
        self._dataset = dataset.lower()
        self.model_path = os.path.join('models', model_filename)
        self.model = keras.models.load_model(self.model_path)
        
        if dataset.lower() == 'china':
            map_path = 'china_mapping.json'
        elif dataset.lower() == 'deutschl':
            map_path = 'deutschl_mapping.json'
        else:
            raise Exception("Error loading json file. Wrong dataset, please check and try again")
        
        with open(map_path, "r") as file:
            self._mappings = json.load(file)
        
        self._start_symbols = ["/"] * SEQUENCE_LENGTH
        
        
    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        '''Melody generator.
            - seed : Melody seed based on time-series music representations.
            - num_steps : Number of steps that we want for the network to generate.
            - max_sequence_length : Max number of steps we want to consider in the seed. 
            - temperature : float value [0-1]. 
        '''
        
        # create seed with start symbols
        seed = seed.split()
        melody = seed #initate melody with seed
        seed = self._start_symbols + seed
        
        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]
        
        for _ in range(num_steps):
            
            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]
            
            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            onehot_seed = onehot_seed[np.newaxis, ...]
            
            # make a prediction
            probabilities = self.model.predict(onehot_seed)
            output_int = self._sample_with_temperature(probabilities, temperature)
            
            # update seed
            seed.append(output_int)
            
            # map int to  our encoding
            output_symbol = [key for key,value in self._mappings.items() if value == output_int][0]
            
            # check whether we're at the end of a melody
            if output_symbol == '/':
                break
                
            # update melody
            melody.append(output_symbol)
        
        return melody
            
            
            
    def _sample_with_temperature(self, probabilities, temperature):
        '''
            Sample index values out of probabilities with temperature distribution.
            The lower the value of temperature, the more predictable(homogeneus) the sample will be
            The higher the value, the more unpredictable the sample will be 
        '''
        
        predictions = np.log(probabilities) / temperature # Normalization
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        choices = range(len(probabilities[0]))
        index = np.random.choice(choices, p=probabilities[0])
        
        return index
    
    def save_melody(self, melody, step_duration=0.25, format='midi', file_name=None):
        
        if file_name == None:
            file_name ='mel_' + self._dataset + '.midi'
        
        # create a music21 stream
        stream = m21.stream.Stream()
        
        # parse all the symbols in the melody and create note/rest objects
        start_symbol = None
        step_counter = 1  #counter to keep track of the note duration
        
        for i, symbol in enumerate(melody):
            
            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):
            
                # ensure we're dealing ith note/rest beyond the first one
                if start_symbol is not None:
                    
                    quarter_length_duration = step_duration * step_counter
                    
                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)
                    
                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
                    
                    stream.append(m21_event)
                    
                    # reset counter
                    step_counter = 1
                    
                start_symbol = symbol
            
            # handle case in which we have a prolongation sign '_'
            else:
                step_counter += 1  
            
        
        # write the m21 stream to a midi file
        stream.write(format, os.path.join(f'midi/{self._dataset}', file_name))
        

In [55]:
if __name__ == '__main__':
    mg = MelodyGenerator(dataset='china', model_filename='model_china_50.h5')
    num_melodies = 50
    for i in range(num_melodies):
        melody = mg.generate_melody(seed=random.choice(china_seeds),
                                    num_steps=500,
                                    max_sequence_length=SEQUENCE_LENGTH,
                                    temperature = 0.4)
        i_str = str(i+1)
        i_filled = i_str.zfill(3)
        mg.save_melody(melody,
                      file_name='mel_china_d4_' + i_filled + '.midi')

<ipython-input-54-dfd818e14ec6>:75: RuntimeWarning: divide by zero encountered in log
  predictions = np.log(probabilities) / temperature # Normalization


In [52]:
del mg